In [3]:
import pandas as pd
import json

In [4]:
lviv_dataset = pd.read_csv("../../data/lviv_static_data/lviv_final_stop_times.csv", encoding="utf-8", sep=",", \
                            header=0, low_memory=False)

In [7]:
stops = lviv_dataset["stop_id"].unique()
journeys = lviv_dataset["trip_id"].unique()

# lvivStops dataset parsing
The final dataset will have such form:
```
[{
   "id": <String> (stop ID),\
   "label": <String> (stop name),
   "connections": <List> (adjacent stops of the current stop), \
   "lat": <Float> (stop latitude),
   "lon":  <Float> (stop longitude),
   "routes": <List> (the routes that are served by the stop),
    "stationType": <List> (types of vehicles))  
},...]

```

In [65]:
### creating the lvivStops.dataset
parsed_stops = dict()
for stop in stops:
    stop_info = lviv_dataset[lviv_dataset["stop_id"] == stop]
    stop_row_indexes = lviv_dataset.index[lviv_dataset["stop_id"] == stop].tolist()
    
    #+++Collecting stop connections with another stops
    connections = set()
    for index in stop_row_indexes:
        if index-1 in lviv_dataset.index and lviv_dataset.loc[index-1]["trip_id"] == stop_info["trip_id"].values[0]:
                connections.add(str(lviv_dataset.loc[index-1]['stop_id']))
        if index+1 in lviv_dataset.index and lviv_dataset.loc[index+1]["trip_id"] == stop_info["trip_id"].values[0]:
            connections.add(str(lviv_dataset.loc[index+1]['stop_id']))

    #+++Collecting routes
    routes = stop_info["route_id"].unique().tolist()
    route_names = stop_info["route_short_name"].unique().tolist()
    
    #+++Collecting station types
    station_type = set()
    for route in route_names:
        if route[0] == "А":
            station_type.add("Bus")
        elif route[0] == "Н-А": 
            station_type.add("Night Bus")
        elif route[:2] == "Тр":
            station_type.add("Trolleybus")
        elif route[0] == "Т":
            station_type.add("Tram")
            
        
    parsed_stops[str(stop_info["stop_id"].values[0])] = {
          "id": str(stop_info["stop_id"].values[0]),\
          "label": stop_info["stop_name"].values[0],
          "connections": list(connections), \
          "lat": stop_info["stop_lat"].values[0],
          "lon":  stop_info["stop_lon"].values[0],
          "routes": [str(route) for route in routes],
          "stationType": list(station_type)               
          }

In [59]:
with open("../../data/lviv_static_data/lvivStops.json", "w") as final_file:
    json.dump(list(parsed_stops.values()), final_file, ensure_ascii=False)

<class 'list'>


# lvivJourneys dataset parsing
The final dataset will have such form:
```
[{
   "id": <String> (stop ID),\
   "label": <String> (stop name),
   "connections": <List> (adjacent stops of the current stop), \
   "lat": <Float> (stop latitude),
   "lon":  <Float> (stop longitude),
   "routes": <List> (the routes that are served by the stop),
    "stationType": <List> (types of vehicles))  
},...]

```